# Advanced Retrieval Strategies - 第二部分
## ReRank

### FlagEmbeddingReranker
這是用於在本地執行開源重新排序（reranker）模型的 **LlamaIndex 整合類別**。

**運作方式：**  
它是對 BAAI（北京智源人工智慧研究院）模型套件的封裝。該類別採用 **Cross-Encoder（交叉編碼器）架構**，也就是同時處理查詢（query）與文件（document），以計算「真正的」相關性分數，而不是比較預先計算好的向量。  

**最適合對象：**  
希望在自有硬體（GPU）上執行高品質重新排序、且不想支付 API 呼叫費用的開發者。

---

**BAAI/bge-reranker-large**

這是目前全球最受歡迎的開源重新排序模型之一。

**主要優勢：**  
它在 MTEB（Massive Text Embedding Benchmark，大規模文字嵌入基準測試）中長期名列前茅。「Large」版本（約 5.6 億參數）在英文與中文任務上都具有極高的準確度。  

**取捨：**  
由於它是 Cross-Encoder 架構，其速度明顯慢於初始的向量搜尋。因此，通常只會將前 10–50 筆文件傳入該模型進行重新排序。

**目前狀態：**  
較新的 **BGE-Reranker-v2-m3**（近期發布）功能更加全面，支援多語言並可處理極長的上下文內容。


In [ ]:
pip install git+https://github.com/FlagOpen/FlagEmbedding.git

In [ ]:
import os

# os.chdir("../")

from llama_index.llms.openai import OpenAI
# from llama_index.indices.postprocessor import LLMRerank 
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

from initialization import credential_init

credential_init()

llm = OpenAI("gpt-4o-mini")
reranker = FlagEmbeddingReranker(
    top_n=10,
    model="BAAI/bge-reranker-large",
    use_fp16=False
)
# reranker = LLMRerank(choice_batch_size=5, top_n=5, llm=llm)

In [ ]:
import wikipediaapi

wiki_titles = ["鋼之鍊金術師", "一拳超人", "ONE_PIECE", "東京喰種"]
wiki_metadatas = {
    "鋼之鍊金術師": {
        "author": "荒川弘",
    },
    "一拳超人": {
        "author": "ONE",
    },
    "ONE_PIECE": {
        "author": "尾田榮一郎",
    },
    "東京喰種": {
        "author": "石田翠",
    },
}

wiki_wiki = wikipediaapi.Wikipedia(user_agent='AI Tutorial(mengchiehling@gmail.com)', language='zh-tw')

從Wikipedia下載數據

In [ ]:
from pathlib import Path

from llama_index.core import SimpleDirectoryReader

data_path = Path("week_3/data")

for title in wiki_titles:
    page = wiki_wiki.page(title)
    wiki_text = page.text

    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path /f"{title}.txt", "w", encoding="utf-8") as fp:
        fp.write(wiki_text)

# Load all wiki documents
docs_dict = {}
for wiki_title in wiki_titles:
    doc = SimpleDirectoryReader(
        input_files=[data_path/f"{wiki_title}.txt"]
    ).load_data()[0]

    doc.metadata.update(wiki_metadatas[wiki_title])
    docs_dict[wiki_title] = doc

documents = [docs_dict[wiki_title] for wiki_title in wiki_titles]

將上週的內容copy/paste下來，建立VectorStoreIndex

In [ ]:
import faiss
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.callbacks import LlamaDebugHandler, CallbackManager

d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

callback_manager = CallbackManager([LlamaDebugHandler()])

index = VectorStoreIndex(
    [],
    storage_context=storage_context,
    transformations=[SentenceSplitter.from_defaults()],
    callback_manager=callback_manager,
    embed_model=embed_model
)

# add documents to index
for wiki_title in wiki_titles:
    index.insert(docs_dict[wiki_title])

In [ ]:
print(len(index.index_struct.nodes_dict))

In [ ]:
rerank_query_engine = index.as_query_engine(
    similarity_top_k=50,
    node_postprocessors=[
        reranker
    ]
)

basic_query_engine = index.as_query_engine(
    similarity_top_k=5,
)

In [ ]:
rerank_response = rerank_query_engine.query("英雄")

In [ ]:
retrieved_nodes = rerank_response.source_nodes

In [ ]:
len(retrieved_nodes)

In [ ]:
retrieved_nodes[:5]

In [ ]:
basic_response = basic_query_engine.query("英雄")

In [ ]:
retrieved_nodes = basic_response.source_nodes

In [ ]:
retrieved_nodes

### GPTReranker

RankGPT 是一種**列表式重排序（Listwise Reranking）**策略，利用大型語言模型（如 GPT-4o 或 Llama 3）來對文件進行排序。

- 運作原理： 它並非逐一為文件評分，而是將查詢（Query）與一整組文件列表同時輸入至大型語言模型（LLM），並詢問：「這些文件中哪些最相關？請按順序輸出編號。」
- 核心優勢： 它能發揮前沿 LLM 的完整推理能力，比起小型的編碼器模型（Encoder-only models），它更能理解語意細微的差別、複雜邏輯以及使用者的意圖。
- 權衡考量： 成本較高（API Token 消耗）且延遲較長（需等待 LLM 生成結果）。它通常被視為高標準檢索流程中的「最後磨光（Final Polish）」步驟。

In [ ]:
# !pip install llama-index-postprocessor-rankgpt-rerank

In [ ]:
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank

reranker = RankGPTRerank(llm=llm)

In [ ]:
rerank_query_engine = index.as_query_engine(
    similarity_top_k=50,
    node_postprocessors=[
        reranker
    ]
)

basic_query_engine = index.as_query_engine(
    similarity_top_k=5,
)

In [ ]:
rerank_response = rerank_query_engine.query("英雄")
retrieved_nodes = rerank_response.source_nodes

In [ ]:
retrieved_nodes[:5]

In [ ]:
basic_response = basic_query_engine.query("英雄")
retrieved_nodes = basic_response.source_nodes

In [ ]:
retrieved_nodes

### Cohere Rerank 4

Cohere Rerank 4 (正式名稱為 Rerank v4.0) 是 Cohere 於 2025 年底推出的最新一代重排序模型。與前代（v3.5）相比，它在效能、上下文長度和企業級應用上都有顯著的突破，被視為目前生產環境中最領先的託管型重排序解決方案之一。

以下是 Rerank 4 的關鍵特性：

1. 雙版本模型策略
Rerank 4 採取了與 LLM 類似的策略，提供了兩款針對不同需求優化的版本：

Rerank 4 Pro: 最強性能版本，專為追求極致準確度的複雜任務設計（如金融、法律、醫療）。在多項基準測試中其排名均位居前二。

Rerank 4 Fast: 效能平衡版本，在維持比 v3.5 更高準確度的同時，大幅降低了延遲（約比 Pro 快 30-40%），適合高流量與即時性需求。

2. 上下文窗口巨大飛躍 (32k Tokens)
從 4k 到 32k： 相比前代 v3.5 的 4096 tokens，Rerank 4 的上下文長度提升了 8 倍。

處理長文件： 這意味著它現在可以一次性「閱讀」約 50 頁的文件，而不需要將文件切成碎片。這對於需要跨章節理解內容的 RAG 系統來說至關重要。

3. 多語言與跨語言能力
支持 100+ 種語言： 在中文、日文、德文、阿拉伯文等主流語言的表現達到了業界領先水平。

跨語言檢索： 它能極其精準地處理「英文查詢、中文文件」這類跨語言的相關性匹配。

4. 結構化數據與推理能力
原生支持 YAML/JSON： 它可以直接對結構化數據進行排序，而不僅僅是純文本。

更強的推理： 針對需要邏輯推理的查詢（例如：需要綜合多個條件的搜尋），Rerank 4 的理解能力顯著優於傳統的編碼器模型，表現更接近小型 LLM，但成本與速度更優。

5. 易於整合
API 接口： 只需幾行程式碼即可接入（支援 LlamaIndex、LangChain 等框架）。

雲端支持： 已整合進 Azure AI Foundry、AWS SageMaker 等主流企業雲平台，滿足安全性與合規性要求。

總結來說： 如果您正在開發一個企業級 RAG 系統，且對多語言支持、長文件處理或金融/醫療數據精準度有很高要求，Cohere Rerank 4 Pro 是目前的最佳選擇；如果您更在意成本與回覆速度，Rerank 4 Fast 則提供了極佳的性價比。

In [ ]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

# api_key = os.environ["COHERE_API_KEY"]
# cohere_rerank = CohereRerank(api_key=api_key, top_n=2)

In [ ]:
CohereRerank?

#### ReRank 策略的比較表

| 方案 | 成本 | 速度 | 適用場景 | 
| --- | --- | --- | --- |
| BGE (Local) | 免費 (需 GPU) | 快 | 隱私要求高、預算有限 |
| RankGPT | 高 (API 消耗) | 慢 | 極致準確度、長文本理解 | 
| Cohere v4 | 中 (按次收費) | 極快 | 企業生產環境、多語言混合 |


剛才我們學習了如何對現有的結果進行『重新排序』，但如果一開始向量搜尋就沒找對內容（例如關鍵字精確匹配失敗），那重新排序也沒救。因此，接下來我們要進入 Hybrid Retriever（混合檢索），學習如何結合傳統搜尋與語意搜尋的優勢...

## Hybrid Retriever

### BM25

為了讓大家專注於 BM25 的機制而非處理中文斷詞的繁瑣配置，接下來的 Hybrid 範例我們將切換至英文資料集（Tech Companies）。

*LlamaIndex 的 BM25在處理中文文本上需要高度客製化* 

In [ ]:
from pathlib import Path

import wikipediaapi


wiki_titles = ["Anthropic", "OpenAI", "XAI_(company)", "Tesla,_Inc."]
wiki_metadatas = {
    "Anthropic": {
        "headquarter": "San Francisco",
        "industry": "Artificial intelligence"
    },
    "OpenAI": {
        "headquarter": "San Francisco",
        "industry": "Artificial intelligence"
    },
    "XAI_(company)": {
        "headquarter": "Palo Alto",
        "industry": "Technology"
    },
    "Tesla,_Inc.": {
        "headquarter": "Austin",
        "industry": "Automotive"
    },
}


wiki_wiki = wikipediaapi.Wikipedia(user_agent='AI Tutorial(mengchiehling@gmail.com)', language='en')

In [ ]:
for title in wiki_titles:
    page = wiki_wiki.page(title)
    wiki_text = page.text

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w", encoding="utf-8") as fp:
        fp.write(wiki_text)

In [ ]:
documents = []

for wiki_title in wiki_titles:
    doc = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()[0]

    doc.metadata.update(wiki_metadatas[wiki_title])
    documents.append(doc)

In [ ]:
from llama_index.core import Document
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

# initialize node parser
splitter = SentenceSplitter(chunk_size=512)

nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
len(nodes)

One option is to create the BM25Retriever directly from nodes, and save to and from disk.

In [ ]:
import Stemmer

from llama_index.retrievers.bm25 import BM25Retriever

# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=5,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

Here, we cover using a BM25Retriever with a docstore to hold your nodes. The advantage here is that the docstore can be remote (mongodb, redis, etc.)

In [ ]:
# initialize a docstore to store nodes
# also available are mongodb, redis, postgres, etc for docstores
from llama_index.core.storage.docstore import SimpleDocumentStore

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=5,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

In [ ]:
# will retrieve context from specific companies
retrieved_nodes = bm25_retriever.retrieve(
    "artiticial intelligence"
)

In [ ]:
len(retrieved_nodes)

In [ ]:
retrieved_nodes[0]

### BM25 Retriever + MetadataFiltering

In [ ]:
# Define metadata filters
from llama_index.core.vector_stores.types import (
    MetadataFilters,
    MetadataFilter,
    FilterOperator,
    FilterCondition,
)

filters = MetadataFilters(
    filters=[
        MetadataFilter(
            key="headquarter",
            value="San Francisco",
            operator=FilterOperator.EQ,
        )
    ],
    condition=FilterCondition.AND,
)

In [ ]:
retrieved_nodes = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=3,
    filters=filters,  # Add filters here
    stemmer=Stemmer.Stemmer("english"),
    language="english",
).retrieve("Hello, world!")

In [ ]:
!pip install matplotlib

### Queries

**Company Background & Positioning**

> Best for testing keyword matching + semantic understanding

- What is OpenAI and what is its core mission?
- How does Anthropic position itself in the AI safety landscape?
- What company is xAI and why was it founded?
- What is Tesla primarily known for besides electric vehicles?

---

**Founders & Key Figures**

> Strong keyword signals (names, dates) with semantic reinforcement

- Who founded OpenAI and when?
- Who are the founders of Anthropic?
- What is Elon Musk’s involvement in xAI?
- How is Elon Musk connected to Tesla?

---

**Mission, Goals, and Philosophy**

> Conceptual questions where dense retrieval excels

- What does OpenAI mean by artificial general intelligence?
- What is Anthropic’s approach to building safe AI systems?
- What problem is xAI trying to solve?
- What is Tesla’s long-term vision for sustainable energy?

---

**Products & Technologies**

> Hybrid retrieval works especially well for mixed factual + semantic queries

- What products or models has OpenAI released?
- What is Anthropic’s Claude model?
- Does xAI develop large language models?
- What technologies does Tesla develop besides cars?

---

**Comparison Queries**

> Ideal for demonstrating QueryFusionRetriever and multi-document reasoning

- How is OpenAI different from Anthropic?
- Compare the goals of OpenAI and xAI.
- How do AI companies like OpenAI and Anthropic differ from Tesla?
- Which of these companies focuses on AI safety?

---

**Advanced / RAG Demo Queries (Recommended)**

> Excellent for end-to-end RetrieverQueryEngine demonstrations

- Which companies in the documents are focused on artificial intelligence research?
- Which organization emphasizes AI safety and alignment?
- What companies were founded by Elon Musk?
- Summarize the missions of OpenAI, Anthropic, xAI, and Tesla.

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

retrieved_nodes = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=5,
    filters=filters,  # Add filters here
    stemmer=Stemmer.Stemmer("english"),
    language="english",
).retrieve("Hello, world!")

for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

## Hybrid Retriever with BM25 + FAISS（BM25 + FAISS 的混合式檢索器）

### 1️⃣ 內容翻譯（繁體中文）

### 使用 BM25 + FAISS 的混合式檢索器
現在我們將結合 **BM25** 與 **FAISS**，同時進行**稀疏檢索（sparse retrieval）**與**稠密檢索（dense retrieval）**。

檢索結果會透過 **QueryFusionRetriever** 進行整合。

透過這個 retriever，我們可以建立一個完整的 **RetrieverQueryEngine**。

---

### 2️⃣ Hybrid Retriever（混合式檢索器）的好處說明

混合式檢索器結合了 BM25 與向量檢索（如 FAISS）的優點，能有效彌補彼此的限制：

- **提升召回率（Recall）**
  - BM25 擅長精確的關鍵字比對
  - FAISS 擅長捕捉語意相近但用詞不同的內容
  - 兩者結合能找回更多「真正相關」的文件

- **兼顧語意與關鍵字精準度**
  - 對於專有名詞、數字、程式碼等，BM25 表現較佳
  - 對於語意相似、同義改寫的查詢，向量檢索更有優勢

- **降低單一檢索方式的盲點**
  - 僅使用 BM25 可能忽略語意相關但未出現關鍵字的內容
  - 僅使用向量檢索可能在精確字詞匹配上表現不足

- **透過 Query Fusion 提升排序品質**
  - QueryFusionRetriever 可以將多種檢索結果加權、融合與重新排序
  - 提供更穩定且高品質的最終結果

---

### 3️⃣ 總結

**Hybrid Retriever（BM25 + FAISS）** 是一種同時結合  
👉 **關鍵字精準匹配** 與  
👉 **語意理解能力** 的檢索策略。

透過 **QueryFusionRetriever** 將結果整合後，再搭配 **RetrieverQueryEngine**，  
可以打造出更準確、更健壯、也更適合實務應用的 RAG（Retrieval-Augmented Generation）系統。

In [ ]:
import faiss
from llama_index.core import StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore

d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
documents = []

for wiki_title in wiki_titles:
    doc = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()[0]

    doc.metadata.update(wiki_metadatas[wiki_title])
    documents.append(doc)

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

# initialize node parser
splitter = SentenceSplitter(chunk_size=512)

nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")


In [ ]:
from llama_index.core.callbacks import LlamaDebugHandler, CallbackManager

callback_manager = CallbackManager([LlamaDebugHandler()])

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex(nodes=nodes, 
                         storage_context=storage_context,
                         callback_manager=callback_manager,
                         embed_model=embed_model)

In [ ]:
import os 

os.chdir("../")

from llama_index.llms.openai import OpenAI

from initialization import credential_init

credential_init()

llm = OpenAI("gpt-4o-mini")

In [ ]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [
        index.as_retriever(similarity_top_k=2),
        BM25Retriever.from_defaults(
            docstore=index.docstore, similarity_top_k=2
        ),
    ],
    num_queries=1,
    use_async=True,
    llm=llm
)

In [ ]:
nodes = retriever.retrieve("Hello, world!")
for node in nodes:
    display_source_node(node, source_length=5000)

**num_queries**：

在這個步驟中，我們會將多個索引融合成一個單一的檢索器（retriever）。這個檢索器同時也會透過產生與原始問題相關的額外查詢來擴充（augment）你的查詢，並彙總所有查詢的結果。

此設定會執行 4 次查詢：一次使用你的原始查詢，並另外產生 3 個新的查詢。

預設情況下，它會使用以下提示（prompt）來產生額外的查詢：

In [ ]:
"""
QUERY_GEN_PROMPT = (
    "You are a helpful assistant that generates multiple search queries based on a "
    "single input query. Generate {num_queries} search queries, one on each line, "
    "related to the following input query:\n"
    "Query: {query}\n"
    "Queries:\n"
)
"""

In [ ]:
retriever = QueryFusionRetriever(
    [
        index.as_retriever(similarity_top_k=2),
        BM25Retriever.from_defaults(
            docstore=index.docstore, similarity_top_k=2
        ),
    ],
    num_queries=4,
    use_async=True,
    llm=llm
)

#### 深度解析：從多個維度捕捉語意（RAG-Fusion）

這裡設置的 num_queries 實際上體現了 RAG-Fusion 的核心思想。

克服查詢的不確定性：原始查詢往往較為簡短或模糊，而 RAG-Fusion 利用 LLM 將單一查詢拆解、改寫成多個不同角度的搜尋指令。

多角度檢索：例如，當使用者問「英雄」時，LLM 可能會產生「英雄定義」、「動漫中的英雄角色」或「英雄相關劇情」等查詢，確保從向量空間中檢索出更全面的上下文。

消除排序偏差：透過產生多個查詢並配合 Reciprocal Rank Fusion (RRF) 演算法，系統能有效減少單一檢索演算法對特定關鍵字的排序偏見，使最終結果更具魯棒性（Robustness）。

In [ ]:
nodes = retriever.retrieve("Hello, world!")

In [ ]:
len(nodes)

Now, we can plug our retriever into a query engine to synthesize natural language responses.

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever)

In [ ]:
QueryFusionRetriever?

### 1. Reciprocal Rerank（RRF）

**模式（Mode）**：`FUSION_MODE.RECIPROCAL_RANK`（預設）

**運作方式（How it works）**：  
這是目前最常見、也最受歡迎的方法。它不會使用 LLM 來對文件進行評分，而是透過一個數學公式，根據文件在多個結果清單中的排名（rank）來重新排序。

**公式（Formula）**：  

$$
score = \sum_{d \in r} \frac{1}{k + rank(d)}
$$

其中，`k` 是一個常數（通常為 60）。

**適用情境（Best for）**：  
通用型的「混合搜尋（Hybrid Search）」，當你希望在向量搜尋與關鍵字搜尋之間取得良好平衡，同時避免額外的 LLM 成本時，非常適合使用。

---

### 2. Relative Score Fusion（相對分數融合）

**模式（Mode）**：`FUSION_MODE.RELATIVE_SCORE`

**運作方式（How it works）**：  
此方法不再關注文件的排名（例如第 1 名、第 2 名），而是直接使用實際的相似度分數。  
它會將來自不同檢索器（通常分數尺度不同）的分數正規化到 0 到 1 的範圍內，接著再對這些分數取平均。

**適用情境（Best for）**：  
當檢索器所提供的信心分數（confidence score）本身就相當可靠時。

---

### 3. Distribution-Based Score Fusion（分佈式分數融合）

**模式（Mode）**：`FUSION_MODE.DIST_BASED_SCORE`

**運作方式（How it works）**：  
這是一種更進階的分數融合方法，會同時考慮結果集中分數的平均值（mean）與標準差（standard deviation）。  
這有助於處理不同檢索器可能產生「分數非常集中」或「分數分佈非常分散」的情況。

---

### 4. Simple Fusion（簡單融合）

**模式（Mode）**：`FUSION_MODE.SIMPLE`

**運作方式（How it works）**：  
這是一種基礎做法，通常只是將結果直接串接（concatenation）或進行簡單加總。  
相較於 RRF，這種方法在實際生產環境中較少使用，但可用於測試基準效能（baseline performance）。


## SumaryIndex Orchestration

此方法透過設定 IndexNode 物件 來運作，其中 obj 欄位可指向以下任一項：

- Query Engine（查詢引擎）
- Retriever（檢索器）
- Query Pipeline（查詢流程管線）
- 另一個 Node（節點）

In [ ]:
import faiss
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore


wiki_titles = ["Anthropic", "OpenAI", "XAI_(company)", "Tesla,_Inc."]
wiki_metadatas = {
    "Anthropic": {
        "headquarter": "San Francisco",
        "industry": "Artificial intelligence"
    },
    "OpenAI": {
        "headquarter": "San Francisco",
        "industry": "Artificial intelligence"
    },
    "XAI_(company)": {
        "headquarter": "Palo Alto",
        "industry": "Technology"
    },
    "Tesla,_Inc.": {
        "headquarter": "Austin",
        "industry": "Automotive"
    },
}

documents = []

for wiki_title in wiki_titles:
    doc = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()[0]

    doc.metadata.update(wiki_metadatas[wiki_title])
    documents.append(doc)

In [ ]:
d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

index = VectorStoreIndex(
    [],
    storage_context=storage_context,
    transformations=[SentenceSplitter.from_defaults()],
    embed_model=embed_model
)

# add documents to index
for document in documents:
    index.insert(document)

In [ ]:
import Stemmer

from llama_index.retrievers.bm25 import BM25Retriever

vector_retriever = index.as_retriever(similarity_top_k=2)

# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore,
    similarity_top_k=2,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

### 組合物件（Composing Objects）

在此步驟中，我們建立 IndexNode。請注意，text 欄位是用來讓**最上層索引（top-level index）**對該節點進行索引的內容。

對於 向量索引（Vector Index），text 會被轉換為 Embedding

對於 關鍵字索引（Keyword Index），text 會被用來進行關鍵字比對

在此範例中，我們使用的是 SummaryIndex。由於它在查詢時總是會取回所有節點，因此在技術上並不需要依賴 text 來進行檢索。



In [ ]:
from llama_index.core import SummaryIndex
from llama_index.core.schema import IndexNode

vector_obj = IndexNode(
    index_id="vector", obj=vector_retriever, text="Vector Retriever"
)

bm25_obj = IndexNode(
    index_id="bm25", obj=bm25_retriever, text="BM25 Retriever"
)

summary_index = SummaryIndex(objects=[vector_obj, bm25_obj])

# 在 LlamaIndex 中使用 `SummaryIndex` 與 `IndexNode` 的混合檢索（Hybrid Retrieval）

![caption](SummaryIndexOchestration.png)

以下筆記總結了我們關於在 LlamaIndex 中使用 **`SummaryIndex` 搭配 `IndexNode`** 來建立**混合檢索系統（Hybrid Retrieval System）**的討論。

---

## 1. 什麼是 `SummaryIndex`？

**`SummaryIndex`**（先前稱為 **List Index**）是一種基礎資料結構，用來以扁平、線性的方式儲存節點（Nodes）。

### 主要特性

- **結構：** 扁平、依序排列的節點清單  
- **檢索邏輯：**  
  - 預設情況下，會檢索索引中儲存的**所有節點**  
  - **不會**執行相似度搜尋  
- **主要目標：**  
  - 全面性的內容整合  
  - 跨所有已索引內容的摘要與總結  

因此，`SummaryIndex` 特別適合用於**聚合（aggregation）、編排（orchestration）與綜合（synthesis）**，而非精準的目標式檢索。

---

## 2. 使用 `IndexNode` 物件進行檢索編排（Retrieval Orchestration）

當你將 **`IndexNode` 物件**傳入 `SummaryIndex` 時，就不再只是單純的文字索引，而是進入了**檢索編排（retrieval orchestration）**的層次。

每個 `IndexNode` 都可以包裝一個 retriever、tool，或是子索引（sub-index）。

### 範例：包裝多個 Retrievers

```python
from llama_index.core.schema import IndexNode
from llama_index.core import SummaryIndex

# 將專用的 retrievers 定義為 IndexNodes
vector_obj = IndexNode(
    index_id="vector",
    obj=vector_retriever,
    text="Vector Retriever"
)

bm25_obj = IndexNode(
    index_id="bm25",
    obj=bm25_retriever,
    text="BM25 Retriever"
)

# 建立 SummaryIndex 作為容器
summary_index = SummaryIndex(objects=[vector_obj, bm25_obj])
```
在這裡，`SummaryIndex` 扮演的是**路由器（router）**的角色，而不是傳統的搜尋索引。

## 3. 執行流程（Execution Workflow）

當你對包含 `IndexNode` 物件的 `SummaryIndex` 發出查詢時，會發生以下流程：

逐步流程說明

1. 選擇（Selection）
    - 索引會取出其所儲存的 IndexNode 項目
    - 通常會選取所有節點
2. 委派（Delegation）
    - 由於每個節點都包含一個 obj，LlamaIndex 會：
    - 取出（unwrap）該物件
    - 對每個 retriever 執行子查詢（sub-query）
3. 混合檢索（Hybrid Retrieval）
    - Vector Retriever → 語意相似度搜尋
    - BM25 Retriever → 關鍵字導向的搜尋
4. 混合綜合（Hybrid Synthesis）
    - LLM 同時接收來自兩種檢索策略的上下文
    - 產生單一、整合後的回應

## 4. 比較：SummaryIndex vs VectorStoreIndex

| 特性          | SummaryIndex | VectorStoreIndex |
| ----------- | ------------ | ---------------- |
| **結構**      | 依序排列的清單      | 向量空間（Embeddings） |
| **檢索行為**    | （通常）檢索所有節點   | 回傳前 k 個最相似的節點    |
| **主要使用情境**  | 摘要與工具編排      | 事實檢索與問答          |
| **搜尋模式**    | 廣泛、混合式執行     | 目標明確的語意搜尋        |
| **LLM 的角色** | 綜合與聚合        | 排序與回答            |

## ⚠️ 重要提醒：SummaryIndex 的使用邊界與痛點
雖然 SummaryIndex 在結合 IndexNode 時展現了強大的編排能力，但開發者必須警惕其擴展性限制：

Context 爆炸風險：SummaryIndex 的預設行為是檢索其下的所有節點。如果你將它用於一般的文檔節點檢索，當文檔數量達到數百甚至數千時，一次查詢就會把所有文本塞入 Context Window，直接導致長度溢出。

API 費用激增：每一次查詢都會強制執行所有 IndexNode 關聯的子路徑（Sub-paths），這會消耗大量的 Token 成本。

最佳實踐：做「將領」而非「士兵」：

不要將 SummaryIndex 用於存儲成千上萬個原始文檔片段。

應該將其視為 Retriever Orchestrator（檢索編排器），用來管理少數幾個（例如 3-5 個）高性能的工具或子索引。這樣它能確保在一次查詢中，同時調用不同的檢索策略（如 Vector、BM25、KG）並進行最終的總結。

### 查詢（Querying）

當我們進行查詢時，所有物件都會被取回，並用來產生節點（nodes），以取得最終答案。

使用 `tree_summarize` 搭配 `aquery()` 可以確保並行（concurrent）執行，並帶來更快的回應速度。

In [ ]:
import os

os.chdir("../")

In [ ]:
from llama_index.llms.openai import OpenAI

from initialization import credential_init

credential_init()

llm = OpenAI("gpt-4o-mini")

query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize", verbose=True,
    llm=llm
)

In [ ]:
response = await query_engine.aquery(
    "Compare the strength between OpenAI and Anthroopic."
)

In [ ]:
response.response

In [ ]:
# response.source_nodes